In [ ]:
import requests,os,time
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(0.5) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return None,call_id

In [2]:
# Importing packages 
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import pprint
import requests,os,re
from time import sleep
from urllib.request import urlopen

In [26]:
# Header
session = requests.session()
session.headers['emails'] = "vhp312@alumni.ku.dk"
session.headers['names'] = "Marcus Bjarup Thøgersen"
session.headers['description'] = "Til brug for eksamen i Social Data Science, KU (https://kurser.ku.dk/course/aØkk08216u/2018-2019)"

# Getting all the links from the tripadvisor page. Stores it in a list called 'links(Marcus) or listen(Jakob)'
links=[]
for nummer in list(range(0, 10*287, 10)):
    sleep(0.5)
    url_nr = 'https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or{}-Restaurant_Tight-Copenhagen_Zealand.html'.format(nummer)
    links.append(url_nr)
#print(links)

In [27]:
print(links)

['https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or0-Restaurant_Tight-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or10-Restaurant_Tight-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or20-Restaurant_Tight-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or30-Restaurant_Tight-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or40-Restaurant_Tight-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or50-Restaurant_Tight-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or60-Restaurant_Tight-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or70-Restaurant_Tight-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurant_Review-g1

In [28]:
# driver = webdriver.Chrome(ChromeDriverManager().install())
def get_info(link):

    driver.get(url)
    sleep(0.5)
    print("Slept 0.5, getting ", link)
    response = requests.get(link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")

        N, L = [], []
        for items in soup.find_all(class_="reviewSelector"):
            try:
                driver.find_element_by_class_name('taLnk ulBlueLinks').click()
                name = items.find(class_="partial_entry").get_text(strip=True)
            except:
                name = ''
            try:
                location = items.find(class_="userLoc").get_text(strip=True)
            except:
                location = ''
            N.append(name)
            L.append(location)

        return N, L

# Loops over all the links in the list of links and appends all the data to a dataframe.   
data = []
for link in links[:3]:
    n, l = get_info(link)
    data.append(pd.DataFrame({'name': n, 'location': l}))

NameError: name 'driver' is not defined

In [ ]:
# Creating a function that searches for the restaurant name, rating (bubble) and reviews count. 
# The function takes an url as input. 


def get_info(link):
    sleep(0.5)
    print("Slept 0.5, getting ", link)
    response = requests.get(link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")
        
        N, L, M, P = [], [], [], []
        for items in soup.find_all(class_="reviewSelector"):
            try:
                name = items.find(class_="partial_entry").get_text(strip=True)
            except:
                name = ''
            try:
                location = items.find(class_="userLoc").get_text(strip=True)
            except:
                location = ''
            try:
                bubble = items.find(class_="ui_bubble_rating")
            except:
                bubble = ''
            try:
                dollars = items.find(class_="restaurants-list")
            except:
                dollars = ''
            P.append(dollars)
            N.append(name)
            L.append(location)
            M.append(bubble)
                
        return N, L, M, P

# Loops over all the links in the list of links and appends all the data to a dataframe.   
data = []
for link in links:
    n, l, m, p = get_info(link)
    data.append(pd.DataFrame({'name': n, 'location': l, 'bubble':m, 'dollars':p}))



Slept 0.5, getting  https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or0-Restaurant_Tight-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or10-Restaurant_Tight-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or20-Restaurant_Tight-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or30-Restaurant_Tight-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or40-Restaurant_Tight-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or50-Restaurant_Tight-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurant_Review-g189541-d1528309-Reviews-or60-Restaurant_Tight-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Re

In [25]:
trip = pd.concat(data)
trip['bubble'][10:]

# start stop and step variables 
start, stop, step = 37, -9, 1
  
# converting to string data type 
trip["bubble"]= trip["bubble"].astype(str) 
  
# slicing to integer
trip["rating"]= trip["bubble"].str.slice(start, stop, step) 
  
trip


,name,location,bubble,dollars,rating
0,Min hustru fandt denne restaurant ved et “tilf...,Denmark,"<span class=""ui_bubble_rating bubble_50""></span>",None,50
1,God beliggenhed \nLidt traditionelt udvalg\nPæ...,"København, Danmark","<span class=""ui_bubble_rating bubble_40""></span>",None,40
2,"Hyggeligt, god mad, venlig betjening, ok prise...","Lolland, Danmark","<span class=""ui_bubble_rating bubble_50""></span>",None,50
3,"Dejlige cocktails, lækket mad, hyggeligt sted,...","Roskilde, Danmark","<span class=""ui_bubble_rating bubble_50""></span>",None,50
4,Kom forbi sent lørdag aften uden reservation.\...,"Region Hovedstaden, Danmark","<span class=""ui_bubble_rating bubble_50""></span>",None,50
5,På trods af en lidt fortumlet start på grund a...,,"<span class=""ui_bubble_rating bubble_40""></span>",None,40
6,Dette er er super hyggelig restaurant med en f...,"Nivå, Danmark","<span class=""ui_bubble_rating bubble_50""></span>",None,50
7,Denne restaurant indeholder begrebet af >hygge...,,"<span class=""ui_bubble_rating bubble_50""></span>",None,50
8,"En god restaurant i indre by, hvor der ellers ...","København, Danmark","<span class=""ui_bubble_rating bubble_40""></span>",None,40
9,Fra først skridt man træder ind i restaurant T...,,"<span class=""ui_bubble_rating bubble_50""></span>",None,50


In [143]:
trip_csv = trip.to_csv(r'C:\Users\jtoft\Downloads\dataframe_tight.csv')